#**TEXT AND WEB ANALYTICS**

##**LAB PRACTICAL 6**

###**MAHIMA MUNJAL** 

###**17CSU098**

In [1]:
import numpy as np 
import pandas as pd 

In [3]:
from google.colab import files
uploaded = files.upload()

Saving abcnews-date-text.csv to abcnews-date-text (2).csv


In [4]:
import io
data = pd.read_csv(io.BytesIO(uploaded['abcnews-date-text.csv']),error_bad_lines=False)
data

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers
...,...,...
1226253,20201231,what abc readers learned from 2020 looking bac...
1226254,20201231,what are the south african and uk variants of ...
1226255,20201231,what victorias coronavirus restrictions mean f...
1226256,20201231,whats life like as an american doctor during c...


In [5]:
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text
documents

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4
...,...,...
1226253,what abc readers learned from 2020 looking bac...,1226253
1226254,what are the south african and uk variants of ...,1226254
1226255,what victorias coronavirus restrictions mean f...,1226255
1226256,whats life like as an american doctor during c...,1226256


In [6]:
print(len(documents))

1226258


In [7]:
documents.isnull().sum()

headline_text    0
index            0
dtype: int64

## Data Preprocessing

In [8]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
stemmer = SnowballStemmer('english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [14]:
doc_sample = documents[documents['index'] == 5000].values[0][0]
print('Original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n Tokenized and Lemmatized document: ')
print(preprocess(doc_sample))

Original document: 
['staffing', 'rail', 'stations', 'would', 'mean', 'ticket', 'hike']


 Tokenized and Lemmatized document: 
['staff', 'rail', 'station', 'mean', 'ticket', 'hike']


In [11]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs

0                         [decid, commun, broadcast, licenc]
1                                         [wit, awar, defam]
2                     [call, infrastructur, protect, summit]
3                                [staff, aust, strike, rise]
4                       [strike, affect, australian, travel]
                                 ...                        
1226253                          [reader, learn, look, year]
1226254                     [south, african, variant, covid]
1226255         [victoria, coronaviru, restrict, mean, year]
1226256          [what, life, like, american, doctor, covid]
1226257    [women, shed, canberra, reskil, unemploy, pandem]
Name: headline_text, Length: 1226258, dtype: object

## Bag of Words

In [15]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [16]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [17]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[5000]

[(13, 1), (237, 1), (1375, 1), (1405, 1), (1430, 1), (1596, 1)]

In [19]:
bow_doc = bow_corpus[5000]
for i in range(len(bow_doc)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc[i][0],dictionary[bow_doc[i][0]], bow_doc[i][1]))

Word 13 ("staff") appears 1 time.
Word 237 ("station") appears 1 time.
Word 1375 ("ticket") appears 1 time.
Word 1405 ("rail") appears 1 time.
Word 1430 ("mean") appears 1 time.
Word 1596 ("hike") appears 1 time.


## TF-IDF

In [20]:
from gensim import corpora,models
tfidf=models.TfidfModel(bow_corpus)

In [21]:
corpus_tfidf=tfidf[bow_corpus]

In [22]:
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5852942020878993),
 (1, 0.38405854933668493),
 (2, 0.5017732999224691),
 (3, 0.5080878695349914)]


## Running LDA using Bag of Words

In [23]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [25]:
for index, score in sorted(lda_model[bow_corpus[5000]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.44285327196121216	 
Topic: 0.028*"govern" + 0.018*"health" + 0.017*"state" + 0.016*"chang" + 0.016*"coronaviru" + 0.013*"hospit" + 0.013*"canberra" + 0.010*"say" + 0.010*"minist" + 0.010*"care"

Score: 0.29999372363090515	 
Topic: 0.059*"polic" + 0.032*"death" + 0.027*"sydney" + 0.016*"shoot" + 0.015*"miss" + 0.015*"investig" + 0.015*"arrest" + 0.014*"melbourn" + 0.013*"royal" + 0.013*"scott"

Score: 0.15714284777641296	 
Topic: 0.034*"elect" + 0.021*"nation" + 0.018*"tasmania" + 0.015*"say" + 0.012*"tasmanian" + 0.010*"darwin" + 0.010*"presid" + 0.009*"vote" + 0.009*"labor" + 0.009*"beach"

Score: 0.014291977509856224	 
Topic: 0.025*"coronaviru" + 0.025*"china" + 0.022*"live" + 0.021*"news" + 0.020*"market" + 0.014*"busi" + 0.011*"australian" + 0.011*"power" + 0.010*"price" + 0.010*"drum"

Score: 0.014289580285549164	 
Topic: 0.015*"worker" + 0.013*"countri" + 0.012*"help" + 0.012*"announc" + 0.011*"commun" + 0.011*"indigen" + 0.010*"work" + 0.010*"pandem" + 0.010*"polit" + 

## Running LDA using TF-IDF

In [26]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word = dictionary, passes = 2, workers=4)

In [28]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5632221698760986	 
Topic: 0.009*"interview" + 0.007*"andrew" + 0.007*"elect" + 0.007*"violenc" + 0.006*"updat" + 0.006*"financ" + 0.005*"liber" + 0.005*"coronaviru" + 0.005*"domest" + 0.005*"inquest"

Score: 0.19738005101680756	 
Topic: 0.012*"countri" + 0.012*"rural" + 0.009*"hour" + 0.008*"govern" + 0.008*"news" + 0.007*"nation" + 0.006*"climat" + 0.006*"stori" + 0.006*"chang" + 0.006*"monday"

Score: 0.15188860893249512	 
Topic: 0.014*"drum" + 0.013*"polic" + 0.009*"charg" + 0.009*"murder" + 0.008*"fatal" + 0.008*"shoot" + 0.007*"driver" + 0.007*"crash" + 0.007*"sport" + 0.007*"arrest"

Score: 0.012503090314567089	 
Topic: 0.014*"coronaviru" + 0.013*"market" + 0.007*"australian" + 0.006*"rise" + 0.006*"price" + 0.006*"australia" + 0.006*"busi" + 0.006*"share" + 0.006*"queensland" + 0.006*"michael"

Score: 0.012501217424869537	 
Topic: 0.009*"guilti" + 0.009*"charg" + 0.009*"assault" + 0.008*"tuesday" + 0.008*"court" + 0.007*"street" + 0.007*"sexual" + 0.007*"abus" + 0.007*